In [140]:
import pandas as pd
import datetime as dt

In [141]:
# import datasets

event_def = pd.read_csv('../Dataset/event_definitions.csv')
fingerhut_combined = pd.read_csv('../Dataset/fingerhut_combined.csv')

KeyboardInterrupt: 

In [ ]:
event_def

,event_name,journey_id,event_definition_id,milestone_number,stage
0,application_phone_approved,1,15,1.0,Apply for Credit
1,application_phone_declined,1,16,NaN,Apply for Credit
2,application_phone_pending,1,17,NaN,Apply for Credit
3,application_web_approved,1,12,1.0,Apply for Credit
4,application_web_declined,1,13,NaN,Apply for Credit
5,application_web_pending,1,14,NaN,Apply for Credit
6,application_web_submit,1,3,NaN,Apply for Credit
7,application_web_view,1,19,NaN,Apply for Credit
8,account_activitation,1,29,3.0,Credit Account
9,account_fraud_review,1,37,NaN,Credit Account


In [257]:
event_dict = {
    'Apply for Credit': [15, 17, 12, 14, 3, 19],
    'Account Activation': [29],
    'Fraud Review': [37],
    'Promotion and Discover': [1, 2, 9, 10, 22, 23, 20, 21, 24],
    'Downpayment': [27, 26, 8, 25],
    'Shopping': [11, 6, 4, 5],
    'Place Order': [18, 7],
    'Order Shipped': [28]
}

In [258]:
# merge event_def (event_definition_id column) with fingerhut_combined (ed_id column column)

fingerhut_combined = pd.merge(fingerhut_combined, event_def, how='left', left_on='ed_id', right_on='event_definition_id')

KeyboardInterrupt: 

In [276]:
# keep only rows where combined_id is between 1 and 1000 (first 1000 customers)

fingerhut_small = fingerhut_combined[(fingerhut_combined['combined_id'] >= 1) & (fingerhut_combined['combined_id'] <= 1000)]

In [277]:
# groupby combined_id and only keep groups that has at least one instance of ed_id == 29 (account_activation) and does not have any instance of ed_id == 16 (application_phone_declined) or ed_id == 13 (application_web_declined)

fingerhut_active_account = fingerhut_small.groupby('combined_id').filter(lambda x: (x['ed_id'] == 29).any() & (x['ed_id'] != 16).all() & (x['ed_id'] != 13).all())

In [278]:
# only keep 'combined_id', 'event_timestamp', 'event_name_y'

fingerhut_active_account = fingerhut_active_account[['combined_id', 'ed_id']]

In [279]:
fingerhut_active_account

,combined_id,ed_id
31,1,2
32,1,4
33,1,11
34,1,4
35,1,4
...,...,...
31905,999,1
31906,999,1
31907,999,5
31908,999,5


In [280]:
# Use the event_dict to create a new column called 'event_type' in fingerhut_active_account. The value of 'event_type' should be the key of the event_dict, which has lists of ed_id as values.

fingerhut_active_account['event_type'] = fingerhut_active_account['ed_id'].map({v: k for k, l in event_dict.items() for v in l})

In [281]:
# remove ed_id column

fingerhut_active_account = fingerhut_active_account.drop(columns=['ed_id'])

# remove duplicates

fingerhut_active_account = fingerhut_active_account.drop_duplicates()

In [282]:
fingerhut_active_account

,combined_id,event_type
31,1,Promotion and Discover
32,1,Shopping
57,1,Apply for Credit
82,1,Account Activation
92,1,Downpayment
...,...,...
31893,999,Place Order
31894,999,Downpayment
31896,999,Account Activation
31905,999,Promotion and Discover


In [283]:
# groupby combined_id, and if the last event_type is not 'Order Shipped', then add new row with event_type 'No Order Made', and also add the combined_id to the new row

fingerhut_active_account = fingerhut_active_account.groupby('combined_id').apply(lambda x: x.append({'event_type': 'No Order Made', 'combined_id': x['combined_id'].iloc[0]}, ignore_index=True) if x['event_type'].iloc[-1] != 'Order Shipped' else x)

# reset index

fingerhut_active_account = fingerhut_active_account.reset_index(drop=True)

/var/folders/6m/88dwrhnx7m3cybxwl1p0rtq40000gn/T/ipykernel_74297/1022833914.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/6m/88dwrhnx7m3cybxwl1p0rtq40000gn/T/ipykernel_74297/1022833914.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/6m/88dwrhnx7m3cybxwl1p0rtq40000gn/T/ipykernel_74297/1022833914.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/6m/88dwrhnx7m3cybxwl1p0rtq40000gn/T/ipykernel_74297/1022833914.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/6m/88dwrhnx7m3cybxwl1p0rtq40000gn/T/ipykernel_74297/1022833914.py:3: FutureWarning:

The frame.append method is depreca

In [284]:
# check for NA values

fingerhut_active_account.isna().sum()

combined_id    0
event_type     0
dtype: int64

In [285]:
fingerhut_active_account['sequence'] = fingerhut_active_account.groupby('combined_id').cumcount() + 1

In [286]:
fingerhut_active_account

,combined_id,event_type,sequence
0,1,Promotion and Discover,1
1,1,Shopping,2
2,1,Apply for Credit,3
3,1,Account Activation,4
4,1,Downpayment,5
...,...,...,...
1509,999,Place Order,3
1510,999,Downpayment,4
1511,999,Account Activation,5
1512,999,Promotion and Discover,6


In [287]:
pivot_fingerhut_active_account = fingerhut_active_account.pivot(index='combined_id', columns='sequence', values='event_type').reset_index(inplace=False)

In [288]:
pivot_fingerhut_active_account = pivot_fingerhut_active_account.fillna('')

In [289]:
pivot_fingerhut_active_account[50:100]

sequence,combined_id,1,2,3,4,5,6,7
50,211,Promotion and Discover,Apply for Credit,Shopping,Downpayment,Place Order,Account Activation,Order Shipped
51,212,Promotion and Discover,Apply for Credit,Shopping,Downpayment,Place Order,Account Activation,Order Shipped
52,216,Apply for Credit,Shopping,Promotion and Discover,Account Activation,Downpayment,Place Order,Order Shipped
53,221,Apply for Credit,Shopping,Promotion and Discover,Account Activation,No Order Made,,
54,224,Shopping,Account Activation,Downpayment,Place Order,Apply for Credit,Promotion and Discover,No Order Made
55,225,Promotion and Discover,Apply for Credit,Account Activation,Place Order,Downpayment,Order Shipped,
56,233,Apply for Credit,Shopping,Promotion and Discover,Account Activation,No Order Made,,
57,234,Apply for Credit,Shopping,Promotion and Discover,Account Activation,No Order Made,,
58,235,Apply for Credit,Shopping,Promotion and Discover,Account Activation,Place Order,Downpayment,Order Shipped
59,236,Account Activation,Apply for Credit,Shopping,Place Order,Downpayment,Promotion and Discover,Order Shipped


In [298]:
import pandas as pd
import plotly.graph_objects as go

def generate_sankey_chart_data(df: pd.DataFrame):
    # list of list: each list is the set of nodes in each tier/column
    column_values = [df[col] for col in df.columns]

    # this generates the labels for the sankey by taking all the unique values
    labels = sum([list(node_values.unique()) for node_values in column_values], [])

    # initializes a dict of dicts (one dict per tier)
    link_mappings = {col: {} for col in df.columns}

    # each dict maps a node to a unique number value
    i = 0
    for col, nodes in zip(df.columns, column_values):
        for node in nodes.unique():
            link_mappings[col][node] = i
            i += 1

    # specifying which columns are serving as sources and which as targets
    source_cols = df.columns[:-1]
    target_cols = df.columns[1:]
    links = []

    # loop to create a list of links in the format [((src, tgt), wt), (), ()...]
    for source_col, target_col in zip(source_cols, target_cols):
        for source, target in zip(df[source_col], df[target_col]):
            links.append(
                (
                    link_mappings[source_col][source],
                    link_mappings[target_col][target],
                    1  # Weight is 1 for counting transitions
                )
            )

    # creating a dataframe with 3 columns: source, target, and weight
    df_links = pd.DataFrame(links, columns=["source", "target", "weight"])

    # generating three lists needed for the sankey visual
    sources = df_links["source"]
    targets = df_links["target"]
    weights = df_links["weight"]

    return labels, sources, targets, weights


# Your DataFrame
df = pivot_fingerhut_active_account.iloc[:, 2:]

# Call the generate_sankey_chart_data function
labels, sources, targets, weights = generate_sankey_chart_data(df=df)


# Map colors to labels
label_colors = {
    'Promotion and Discover': 'pink',
    'Apply for Credit': 'green',
    'Shopping': 'blue',
    'Account Activation': 'cyan',
    'Downpayment': 'purple',
    'Place Order': 'yellow',
    'Order Shipped': 'orange',
    'No Order Made': 'red',
    '': 'black'  # Adjust this for labels with empty strings
}

# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color=[label_colors[label] for label in labels]  # Map colors based on the dictionary
    ),
    link=dict(
        source=sources,
        target=targets,
        value=weights
    )
)])

fig.update_layout(title_text="Sankey Diagram", font_size=10)
fig.show()


Key points:

- A large proportion of customers exposed to the promotion and discover stage move onto: Apply for Credit, Account Activation, Downpayment
- A large proportion of customers move onto Downpayment or Place Order after Account Activation, which may indicate that Account Activation is a significant factor to the customer's decision to purchase. This will require further investigation and hypothesis testing. If it holds to be true then we can focus more on increaseing Account Activation rate than other stages.

In [48]:
df_sample

,Column1,Column2,Column3,LinkWeight
0,A,X,P,10
1,B,Y,Q,15
2,A,X,P,5
3,C,Z,R,20
4,B,Y,Q,8
5,A,X,R,12
